In [1]:
#! /usr/bin/env python

In [1]:
import jupyros
import rospy
import os
from move_base_msgs.msg import MoveBaseActionFeedback
from move_base_msgs.msg import MoveBaseActionGoal
from nav_msgs.msg import Odometry
from geometry_msgs.msg import Twist
from actionlib_msgs.msg import GoalID
from actionlib_msgs.msg import GoalStatusArray
import sys, tty
from sensor_msgs.msg import LaserScan
from IPython.core.display import display, HTML
from IPython.display import display,clear_output
from ipywidgets import AppLayout, Button, Layout, ButtonStyle, GridBox, VBox, HBox , widgets , interact, interactive, fixed, interact_manual, interactive_output
import time
from jupyros import ros3d

In [2]:
def f(a):
    global x
    try:
        x = float(a.value)
    except:
        print("Please try again")
        
    
def c(a):
    global y
    try:
        y = float(a.value)
    except:
        print("Please try again")
        
def ex(a):
    global exit
    exit = False
    print("Exiting")
    time.sleep(1)
    into()
    
def sub(a):
    global x,y,plan_bol
    try:
        float(x)
        float(y)
        print("Perfect, starting the plan")
        time.sleep(1)
        clear_output(wait=True)
        plan_bol = True
        
    except:
        print("Please try again")
    else:
        into()

    
        
    
def move_base_show():
         
       global x,y
       x = ""
       y= ""
       count_target = []
       #clear_output(wait=True)
       display(HTML('<h1 style = "position:relative; margin-left:300px;">      PLANNING MOTION INTERFACE</h1>'))
       print('\n')
       print('\n')
       display(HTML('<h3 style = "position:relative; margin-left:0px;"> Insert coordinates, enter and then submit</h3>'))
       
       a = widgets.Text(description='x') 
       b = widgets.Text(description='y') 


    
       coo_box = HBox([widgets.VBox([a, b])])
       display(coo_box)

       a.on_submit(f)
       b.on_submit(c)
        
       submit = Button(description='Submit', layout=Layout(width='100', margin= '0px 0px 0px 700px'), style=ButtonStyle(button_color='lightblue'))
       display(submit)
       exit = Button(description='Exit', layout=Layout(width='100', margin= '0px 0px 50px 700px'), style=ButtonStyle(button_color='red'))
       display(exit)
       submit.on_click(sub)
       exit.on_click(ex)
       
move_base_show()

Button(description='Submit', layout=Layout(margin='0px 0px 0px 700px', width='100'), style=ButtonStyle(button_…

Button(description='Exit', layout=Layout(margin='0px 0px 50px 700px', width='100'), style=ButtonStyle(button_c…

In [7]:
def todist(x,y,x_pos,y_pos):
    return ((x-x_pos)**2-(y-y_pos)**2)**0.5

def move_base():

        pub_movebase = rospy.Publisher("/move_base/goal", MoveBaseActionGoal, queue_size = 50)
        rate = rospy.Rate(10)
        move = MoveBaseActionGoal()
        move.goal.target_pose.header.frame_id = "map"
        move.goal.target_pose.pose.orientation.w = 1
        position = rospy.wait_for_message("/odom",Odometry)


        #-------------------------------------
        

        count_target = []
        
        while(True):
            
                # headings
                clear_output(wait = True)
                display(HTML('<h3 style = "position:relative; margin-left:0px;"> Feedback</h3>'))
                feedbackk = widgets.Output(layout={'border': '1px solid black'},style = "position:relative; margin-left:2000px;") 
                display(feedbackk)
                display(HTML('<h3 style = "position:relative; margin-left:0px;"> Distance to goal</h3>'))
                dtg = widgets.Output(layout={'border': '1px solid black'},style = "position:relative; margin-left:2000px;") 
                display(dtg)
                
                #coordinates
                
                display(HTML('<h3 style = "position:relative; margin-left:0px;">Coordinates:</h3>'))
                

                global x,y
                
                
                
                move.goal.target_pose.pose.position.x = x
                move.goal.target_pose.pose.position.y = y
                print(x,y)
                
                
                pub_movebase.publish(move)
                rate.sleep()
                
                goal_info = None
                while(goal_info is None):
                        goal_info = rospy.wait_for_message("move_base/status",GoalStatusArray)
                        rate.sleep()
                status = int(goal_info.status_list[-1].status)  
                            
                while(True):
        
                        goal_info = rospy.wait_for_message("move_base/status",GoalStatusArray)
                        _id = goal_info.status_list[0].goal_id.id
                        status = int(goal_info.status_list[-1].status)
                    
                    
                        position = rospy.wait_for_message("/odom",Odometry)
                        x_pos = position.twist.twist.linear.x 
                        y_pos = position.twist.twist.linear.y 
                        
                        #add Feedback distance, handle it , and print in text box
                        
                        dist = todist(x,y,x_pos,y_pos)
                        
                        #----------------------------------------------
                        if(status == 1):
                            
                            #print in the text boxes
                            with feedbackk:
                                print('\r                   ', end='', flush=True)
                                print(f'\rGoal Accepted', end='', flush=True)
                            with dtg:
                                print('\r                   ', end='', flush=True)
                                print(f'\r{dist}', end='', flush=True)
                            pass
                        #----------------------------------------------
                        elif(status == 2):
                            
                            #add 0 to count due target failed
                            count_target.append(0)
                            
                            #print in the text boxes
                            with feedbackk:
                                print('\r                   ', end='', flush=True)
                                print(f'\rGoal not reachable', end='', flush=True)
                                
                            continue
                        
                        #----------------------------------------------                       
                                                
                        elif(status == 3):
                            
                            #add 1 due target reached
                            count_target.append(1)
                            
                            #print in the text boxes
                            with feedbackk:
                                print('\r                   ', end='', flush=True)
                                print(f'\rGoal reached', end='', flush=True)
                            
                             
                             
                            print('\n')
                            
                            # planning again 
                            
                            button = state_show()
       
                            if(button == 'y'):
                                clear_output(wait=True)
                                move_base_show()
                                break
                    
                            if(button == 'n'):
                                publisher_cancel = rospy.Publisher("/move_base/cancel", GoalID, queue_size = 1)
                                cancel = GoalID()
                                publisher_cancel.publish(cancel)
                                clear_output(wait=True)
                                return 1
                            else:
                                with feedbackk:
                                    print('\r                   ', end='', flush=True)
                                    print(f'\rCommand wrong', end='', flush=True)
                            
                             #----------------------------------------------

                                
                        
                        elif(status == 4):
                            
                            #add 0 due target aborted
                            
                            count_target.append(0)
                            
                            #print in the text boxes
                            with feedbackk:
                                print('\r                   ', end='', flush=True)
                                print(f'\rGoal Cancelled - No plan avaible', end='', flush=True)
                            
                            
                            print('Exiting')
                            time.sleep(1)
                            
                            # planning again
                            move_base_show()
                            
                            """button = state_show()
       
                            if(button == 'y'):
                                clear_output(wait=True)
                                move_base_show()
                                break

                            if(button == 'n'):
                                publisher_cancel = rospy.Publisher("/move_base/cancel", GoalID, queue_size = 1)
                                cancel = GoalID()
                                publisher_cancel.publish(cancel)
                                clear_output(wait=True)
                                return 1
                            
                            
                            else:
                                with feedbackk:
                                    print('\r                   ', end='', flush=True)
                                    print(f'\r{word}', end='', flush=True)"""
                            #----------------------------------------------------
                        
                        rate.sleep()

                        
                        """ 
                        TODO:     -COUNT PLOT (already did)
                                  - test planning again
                                  - insert plot when user decide to no plan again (maybe with a function and three buttons)
                        """
move_base()

KeyboardInterrupt: 

In [8]:
def fforward(forward):
    
    [my_vel.linear.x, my_vel.angular.z] =[1 , 0] 
    vel_pub.publish(my_vel)

def fleft(left):
    [my_vel.linear.x, my_vel.angular.z] =[0 , 2]
    vel_pub.publish(my_vel)
    
def fright(right):
    [my_vel.linear.x, my_vel.angular.z] =[0, -2]
    vel_pub.publish(my_vel)
    
def fbehind(behind):
    [my_vel.linear.x, my_vel.angular.z] =[-1 , 0]
    vel_pub.publish(my_vel)

def fquit(quit):
    global exit
    exit = not exit
    [my_vel.linear.x, my_vel.angular.z] =[0 , 0]
    vel_pub.publish(my_vel)
    into()

    
    

def manual_show():
        
        global my_vel,vel_pub,exit, manual_bol
        vel_pub = rospy.Publisher('/cmd_vel', Twist, queue_size = 50)
        my_vel = Twist()
        
        display(HTML('<h1 style = "position:relative; margin-left:340px;">Motion Planner Interface</h1>'))
        print('\n')
        print('\n')

        """ 
        v = ros3d.Viewer()
        rc = ros3d.ROSConnection(url="ws://localhost:9090") 
        tf_client = ros3d.TFClient(ros=rc, fixed_frame='map')
        laser_view = ros3d.LaserScan(topic="/scan", ros=rc, tf_client=tf_client) 
        g = ros3d.GridModel()
        v.objects = [g, laser_view]
        display(v)"""

        forward = Button(description='^',layout=Layout(width='auto', grid_area='forward'), style=ButtonStyle(button_color='lightblue'))
        left = Button(description='<', layout=Layout(width='auto', grid_area='left'), style=ButtonStyle(button_color='lightblue'))
        right = Button(description='>', layout=Layout(width='auto', grid_area='right'), style=ButtonStyle(button_color='lightblue'))
        behind = Button(description='v', layout=Layout(width='auto', grid_area='behind'), style=ButtonStyle(button_color='lightblue'))
        quit = Button(description='quit', layout=Layout(width='auto', grid_area='quit'), style=ButtonStyle(button_color='red'))

        box = GridBox(children=[forward,left,right,behind,quit], layout=Layout(width='auto',grid_template_rows='auto auto auto', grid_template_columns='auto auto auto', grid_template_areas='''
        " . forward . . ." 
        "left . right . ."
        " . behind . . quit "
        ''')
        )   
        display(box)
        forward.on_click(fforward)
        left.on_click(fleft)
        right.on_click(fright)
        behind.on_click(fbehind)
        quit.on_click(fquit)






manual_show()



GridBox(children=(Button(description='^', layout=Layout(grid_area='forward', width='auto'), style=ButtonStyle(…

In [9]:
print(exit)

None


In [10]:
#Come uscire fuori?

def manual():
    

    manual_bol = False
    exit = True
    manual_show()
    


    


        

            


manual()

GridBox(children=(Button(description='^', layout=Layout(grid_area='forward', width='auto'), style=ButtonStyle(…

In [11]:
def clbk_laser(msg):
        min_fright = min(min(msg.ranges[144:287]), 5)
        min_front =  min(min(msg.ranges[288:431]), 5)
        min_fleft =  min(min(msg.ranges[432:575]), 5)
        
        return min_fright, min_front, min_fleft

In [12]:
def take_action(min_fright,min_front, min_fleft):
    msg = Twist()
    msg.linear.y = 0
    msg.linear.z = 0
    msg.angular.x = 0
    msg.angular.y= 0
    my_vel = rospy.Publisher('/cmd_vel', Twist, queue_size = 50)
    state_description = ''
    if min_front > 1 and min_fleft > 1 and min_fright > 1:
        state_description = 'case 1 - nothing'
        linear_x = 0.6
        angular_z = 0
    elif min_front < 1 and min_fleft > 1 and min_fright > 1:
        state_description = 'case 2 - front'
        linear_x = -2
        angular_z = 0
    elif min_front > 1 and min_fleft > 1 and min_fright < 1:
        state_description = 'case 3 - fright'
        linear_x = 0
        angular_z = 1
    elif min_front > 1 and min_fleft < 1 and min_fright > 1:
        state_description = 'case 4 - fleft'
        linear_x = 0
        angular_z = -1
    elif min_front < 1 and min_fleft > 1 and min_fright < 1:
        state_description = 'case 5 - front and fright'
        linear_x = 0
        angular_z = 1
    elif min_front < 1 and min_fleft < 1 and min_fright > 1:
        state_description = 'case 6 - front and fleft'
        linear_x = 0
        angular_z = -1
    elif min_front < 1 and min_fleft < 1 and min_fright < 1:
        state_description = 'case 7 - front and fleft and fright'
        linear_x = 0
        angular_z = 2
    elif min_front > 1 and min_fleft < 1 and min_fright < 1:
        state_description = 'case 8 - fleft and fright'
        linear_x = 0.3
        angular_z = 0

    #rospy.loginfo(state_description)
    #print in the text box of motion planning
    msg.linear.x = linear_x
    msg.angular.z = angular_z
    my_vel.publish(msg)
        

In [27]:
def assisted_manual_show():
    
        
        #attivare i bottoni solo se lontano dagli ostacoli
        #in every button if near obstacles take_action() other wise directly publish
        display(HTML('<h1 style = "position:relative; margin-left:370px;">Assisted Motion Planner</h1>'))
        print('\n')
        print('\n')
        from jupyros import ros3d
        v = ros3d.Viewer()
        rc = ros3d.ROSConnection(url="ws://localhost:9090") 
        tf_client = ros3d.TFClient(ros=rc, fixed_frame='map')
        laser_view = ros3d.LaserScan(topic="/scan", ros=rc, tf_client=tf_client) 
        g = ros3d.GridModel()
        v.objects = [g, laser_view]
        display(v)
        
        
        display(HTML('<h3 style = "position:relative; margin-left:0px;">Closest obstacle distance:</h3>'))
        print('\n')
        assisted_obstacle = widgets.Output(layout={'border': '1px solid black'},style = "position:relative; margin-left:2000px;") 
        display(assisted_obstacle)
        display(HTML('<h3 style = "position:relative; margin-left:0px;">State:</h3>'))
        print('\n')
        state = widgets.Output(layout={'border': '1px solid black'},style = "position:relative; margin-left:2000px;") 
        display(state)
        print('\n')

        forward = Button(description='^',layout=Layout(width='auto', grid_area='forward'), style=ButtonStyle(button_color='lightblue'))
        left = Button(description='<', layout=Layout(width='auto', grid_area='left'), style=ButtonStyle(button_color='moccasin'))
        right = Button(description='>', layout=Layout(width='auto', grid_area='right'), style=ButtonStyle(button_color='salmon'))
        behind = Button(description='v', layout=Layout(width='auto', grid_area='behind'), style=ButtonStyle(button_color='salmon'))
        quit = Button(description='quit', layout=Layout(width='auto', grid_area='quit'), style=ButtonStyle(button_color='red'))
        field = GridBox(children=[forward,left,right,behind,quit], layout=Layout(width='auto',grid_template_rows='auto auto auto', grid_template_columns='auto auto auto', grid_template_areas='''
        " . forward ." 
        "left . right"
        " . behind . "
        ''')
        )
        display(field)
        return state, assisted_obstacle

state, assisted_obstacle = assisted_manual_show()
    


Viewer(objects=[GridModel(), LaserScan(ros=ROSConnection(url='ws://localhost:9090'), tf_client=TFClient(fixed_…

Output(layout=Layout(border='1px solid black'))

Output(layout=Layout(border='1px solid black'))

GridBox(children=(Button(description='^', layout=Layout(grid_area='forward', width='auto'), style=ButtonStyle(…

In [30]:
#Advertise the assisted motion interface that an obstacle is near
def assisted_print(word):
    with assisted_obstacle:
        print(f'\r{word}', end='', flush=True)

        

word = "state: moving"
assisted_print(word)
        

In [15]:
def assisted_manual():
        os.system("clear")
        assisted_manual_show()
        vel_pub = rospy.Publisher("cmd_vel", Twist, queue_size= 50)
        
        my_vel = Twist()
        my_vel.linear.y = 0
        my_vel.linear.z = 0
        my_vel.angular.x = 0
        my_vel.angular.y= 0
        
        
        while True:
        
                laser = rospy.wait_for_message("/scan", LaserScan)     
                vel_pub = rospy.Publisher("cmd_vel", Twist, queue_size= 50)
                
                [min_fright, min_front, min_fleft] = clbk_laser(laser)
                
                
                if min_front < 1 or min_fleft < 1 or min_fright < 1:
                        take_action(min_fright, min_front, min_fleft)
                        
                else:    
                        arrow = input()
                        if (arrow == 'w'):
                                [my_vel.linear.x, my_vel.angular.z] =[1 , 0] 
                        elif (arrow == 'a'):
                                [my_vel.linear.x, my_vel.angular.z] =[0 , 2]
                        elif (arrow == 'd'):
                                [my_vel.linear.x, my_vel.angular.z] =[0, -2]
                        elif (arrow == 's'):
                                [my_vel.linear.x, my_vel.angular.z] =[-1 , 0]
                        elif (arrow == 'q'):
                                [my_vel.linear.x, my_vel.angular.z] =[0 , 0]
                                vel_pub.publish(my_vel)
                                break
                        else:
                                print("Wrong button")
                                
                        vel_pub.publish(my_vel)
                        
        return 1

In [16]:
def man(b):

    global manual_bol,exit
    manual_bol = True
    exit = True
    into()
    
def pla(b):
    global plan_bol
    plan_bol_bol = True
    into()
    
    
def amm(b):
    assisted_manual()
    into()
    
def intro():
        os.system("clear")
        display(HTML('<h1 style = "position:relative; margin-left:460px;">      MENU</h1>'))
        print('\n')
        print('\n')
        plan = Button(description='Planning',layout=Layout(width="1000px",height= "100px", grid_area='forward'), style=ButtonStyle(button_color='lightblue'))
        manual_motion = Button(description='Manual Motion',layout=Layout(width="1000px",height= "100px", grid_area='forward'), style=ButtonStyle(button_color='lightblue'))
        assisted_manual_motion = Button(description=' Assisted Manual Motion',layout=Layout(width="1000px",height= "100px", grid_area='forward'), style=ButtonStyle(button_color='lightblue'))

        boxI = VBox([plan,manual_motion, assisted_manual_motion])
        manual_motion.on_click(man)
        plan.on_click(pla)
        assisted_manual_motion.on_click(amm)
        display (boxI)


intro()

In [76]:

def into():
        #clear_output(wait=True)
        global manual_bol, exit, plan_bol
        print(exit)

             
        if(manual_bol):
            manual_bol = False
            print(manual_bol)
            clear_output(wait=True)
            manual_show()
            #into()
        elif(plan_bol):
            plan_bol = False
            move_base_show()
            
        elif(not exit):
            exit = True
            main()

    
def main():
        global exit, plan_bol,manual_bol
        manual_bol = None
        plan_bol = None
        exit = None
        
        clear_output(wait=True)
        rospy.init_node('controller')
        intro()
        
        
        #while (not rospy.is_shutdown() ):
                
                #intro()
                #command = int(input("Insert choice"))

        #into()
                

        """if(command == 1):
                                clear_output(wait=True)
                                move_base()
                                break
                        
                        if(command == 2):
                                    global giuda 
                                    giuda = True
                                    global exit
                                    exit = True
                                    while(exit):

                                        if(giuda):   
                                            clear_output(wait=True)
                                            manual()
                                        time.sleep(0.1)
                                            
                                    print("Uscito")
                                 
                           
                        
                        if(command == 3):
                                clear_output(wait=True)
                                assisted_manual()
                                break
                        
                        if(command == 4):
                                clear_output(wait=True)
                                return 1
                   
                        
                        else:
                                clear_output(wait=True)
                                print("Command incorrect")
                                break
                        
                        time.sleep(10)"""
main()

GridBox(children=(Button(description='^', layout=Layout(grid_area='forw', width='auto'), style=ButtonStyle(but…

Output(layout=Layout(border='1px solid black'))

Output(layout=Layout(border='1px solid black'))

Button(description='Plot Target', layout=Layout(margin='0px 0px 0px 700px', width='100'), style=ButtonStyle(bu…

Button(description='Exit', layout=Layout(margin='0px 0px 0px 700px', width='100'), style=ButtonStyle(button_co…

ROSInterruptException: rospy shutdown

In [18]:
print(exit)

None


In [71]:
def forw(forward):
    [my_vel.linear.x, my_vel.angular.z] =[1 , 0] 
    vel_pub.publish(my_vel)

def lef(left):
    [my_vel.linear.x, my_vel.angular.z] =[0 , 2]
    vel_pub.publish(my_vel)
    
def rig(right):
    [my_vel.linear.x, my_vel.angular.z] =[0, -2]
    vel_pub.publish(my_vel)
    
def beh(behind):
    [my_vel.linear.x, my_vel.angular.z] =[-1 , 0]
    vel_pub.publish(my_vel)

def quit(quit):
    global exit
    exit = False
    [my_vel.linear.x, my_vel.angular.z] =[0 , 0]
    vel_pub.publish(my_vel)
    into()

    
    

def manual_show():
    
        global my_vel,vel_pub,exit, manual_bol
        vel_pub = rospy.Publisher('/cmd_vel', Twist, queue_size = 50)
        my_vel = Twist()
        
        display(HTML('<h1 style = "position:relative; margin-left:340px;">Motion Planner Interface</h1>'))
        print('\n')
        print('\n')

        forward = Button(description='^',layout=Layout(width='auto', grid_area='forward'), style=ButtonStyle(button_color='lightblue'))
        left = Button(description='<', layout=Layout(width='auto', grid_area='left'), style=ButtonStyle(button_color='lightblue'))
        right = Button(description='>', layout=Layout(width='auto', grid_area='right'), style=ButtonStyle(button_color='lightblue'))
        behind = Button(description='v', layout=Layout(width='auto', grid_area='behind'), style=ButtonStyle(button_color='lightblue'))
        quit = Button(description='quit', layout=Layout(width='auto', grid_area='quit'), style=ButtonStyle(button_color='red'))

        box = GridBox(children=[forward,left,right,behind,quit], layout=Layout(width='auto',grid_template_rows='auto auto auto', grid_template_columns='auto auto auto', grid_template_areas='''
        " . forward . . ." 
        "left . right . ."
        " . behind . . quit "
        ''')
        )   
        
        display(box)
        forward.on_click(fforward)
        left.on_click(fleft)
        right.on_click(fright)
        behind.on_click(fbehind)
        quit.on_click(fquit)






manual_show()



GridBox(children=(Button(description='^', layout=Layout(grid_area='forward', width='auto'), style=ButtonStyle(…

In [89]:
def plt(a):
    print("ok")
    
def frw(a):
    print('ok')
    
def be(a):
    print('ok')
    
def le(a):
    print('ok')
    
def ri(a):
    print('ok')
    
def exm(a):
    print('ok')
    
    
def assisted_manual_show():
    
        
        #attivare i bottoni solo se lontano dagli ostacoli
        #in every button if near obstacles take_action() other wise directly publish
        display(HTML('<h1 style = "position:relative; margin-left:370px;">Assisted Motion Planner</h1>'))
        print('\n')
        print('\n')
        from jupyros import ros3d

        forw = Button(description='^',layout=Layout(width='auto', grid_area='forw'), style=ButtonStyle(button_color='lightblue'))
        lef = Button(description='<', layout=Layout(width='auto', grid_area='lef'), style=ButtonStyle(button_color='lightblue'))
        rig = Button(description='>', layout=Layout(width='auto', grid_area='rig'), style=ButtonStyle(button_color='lightblue'))
        beh = Button(description='v', layout=Layout(width='auto', grid_area='beh'), style=ButtonStyle(button_color='lightblue'))

        field = GridBox(children=[forw,lef,rig,beh], layout=Layout(width='auto',grid_template_rows='auto auto auto', grid_template_columns='auto auto auto', grid_template_areas='''
        " . forw . " 
        "lef . rig "
        " . beh . "
        ''')
        )
        
        display(field)
        print('\n')
        display(HTML('<h3 style = "position:relative; margin-left:0px;">Closest obstacle distance:</h3>'))
        print('\n')
        min_assfront = widgets.Output(layout={'border': '1px solid black'},style = "position:relative; margin-left:2000px;") 
        display(min_assfront)
        display(HTML('<h3 style = "position:relative; margin-left:0px;">State:</h3>'))
        print('\n')
        state = widgets.Output(layout={'border': '1px solid black'},style = "position:relative; margin-left:2000px;") 
        display(state)
        print('\n')
        
        plt_pos = Button(description='Plot position', layout=Layout(width='100', margin= '0px 0px 0px 700px'), style=ButtonStyle(button_color='orange'))
        display(plt_pos)
        
        exitm = Button(description='Exit', layout=Layout(width='100', margin= '0px 0px 0px 700px'), style=ButtonStyle(button_color='red'))
        display(exitm)
        
        exitm.on_click(exm)
        plt_pos.on_click(plt)
        forw.on_click(frw)
        lef.on_click(le)
        rig.on_click(ri)
        beh.on_click(be)
        return state,min_assfront

state, min_assfront = assisted_manual_show()
    


GridBox(children=(Button(description='^', layout=Layout(grid_area='forw', width='auto'), style=ButtonStyle(but…

Output(layout=Layout(border='1px solid black'))

Output(layout=Layout(border='1px solid black'))

Button(description='Plot position', layout=Layout(margin='0px 0px 0px 700px', width='100'), style=ButtonStyle(…

Button(description='Exit', layout=Layout(margin='0px 0px 0px 700px', width='100'), style=ButtonStyle(button_co…

In [90]:
with min_assfront:
    print("cazzo guardi")